This file has the code to visualise syllable transition values and create a graph based on it

Date: 20/9/2022

In [1]:
import math
import pandas
import os
import numpy as np
import matplotlib.pyplot as plt

from markovchain import MarkovChain

In [14]:
file = pandas.read_csv('M09_trans_prob_processed.csv', header=None)

#Drop the row and column labels, just keep the numeric values
trans_prob = file.iloc[1:, 1:]
trans_prob = np.array(trans_prob)

#Get the set of unique syllables for the individual with their own index (so convert array to list)
uniq_syl = file.iloc[:1, 1:]
uniq_syl = np.array(uniq_syl)
#uniq_syl

In [ ]:
#Doesnt work for more than 4 states
mc = MarkovChain(trans_prob, uniq_syl)